In [4]:
import ee_s1_ard
import ee 
ee.Authenticate()
ee.Initialize(project='ee-caiosimplicioarantes') 


In [6]:
from ee_s1_ard import S1ARDImageCollection

In [21]:
import zipfile
import geopandas as gpd
import logging
import os
import locale
import requests

#from ravi_ee_tools import load_aoi_from_shapefile # Assuming your_module_name.py contains the function
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
app_logger = logging.getLogger(__name__)
logger = logging.getLogger(__name__)

In [93]:

def load_aoi_from_shapefile(shapefile_path):
    """
    Loads an Area of Interest (AOI) from a shapefile (or a zip containing a shapefile)
    and converts it into an Earth Engine FeatureCollection.

    Args:
        shapefile_path (str): The path to the shapefile (.shp) or a zip archive
                              containing a .shp file.

    Returns:
        ee.FeatureCollection: An Earth Engine FeatureCollection representing the AOI.

    Raises:
        FileNotFoundError: If no .shp file is found inside a zip archive.
        ValueError: If the shapefile does not contain any geometries.
        Exception: For other errors during shapefile reading or processing.
    """
    gpd_aoi = None # Use a distinct name for the geopandas DataFrame

    if shapefile_path.endswith(".zip"):
        with zipfile.ZipFile(shapefile_path, "r") as zip_ref:
            shapefile_within_zip = None
            for file in zip_ref.namelist():
                if file.lower().endswith(".shp"): # Use .lower() for case-insensitivity
                    shapefile_within_zip = file
                    break
            if not shapefile_within_zip:
                logger.error(f"No .shp file found inside the zip archive: {shapefile_path}")
                raise FileNotFoundError(
                    f"No .shp file found inside the zip archive: {shapefile_path}"
                )

            # Read shapefile directly from the zip archive.
            gpd_aoi = gpd.read_file(f"zip://{shapefile_path}/{shapefile_within_zip}")
    else:
        gpd_aoi = gpd.read_file(shapefile_path)

    # Reproject the GeoDataFrame to EPSG:4326 to ensure correct
    # coordinates for Earth Engine.
    gpd_aoi = gpd_aoi.to_crs(epsg=4326)

    if gpd_aoi.empty:
        logger.error(f"The shapefile at {shapefile_path} does not contain any geometries.")
        raise ValueError(f"The shapefile at {shapefile_path} does not contain any geometries.")

    # Dissolve multiple features into a single geometry if necessary.
    if len(gpd_aoi) > 1:
        logger.info("Multiple features found; dissolving into a single geometry.")
        gpd_aoi = gpd_aoi.dissolve()

    # Extract the first geometry.
    # It's safer to ensure there's at least one geometry before iloc[0]
    if gpd_aoi.empty: # Re-check after dissolve if it became empty (unlikely but robust)
         logger.error(f"The shapefile at {shapefile_path} became empty after dissolve.")
         raise ValueError(f"The shapefile at {shapefile_path} became empty after dissolve.")

    geometry = gpd_aoi.geometry.iloc[0]

    # Convert the geometry to GeoJSON format.
    geojson = geometry.__geo_interface__

    # Remove any third dimension from the coordinates.
    # Note: GeoJSON can support 3D, but Earth Engine often expects 2D.
    # This part of the logic is robust.
    if geojson["type"] == "Polygon":
        geojson["coordinates"] = [
            list(map(lambda coord: coord[:2], ring)) for ring in geojson["coordinates"]
        ]
    elif geojson["type"] == "MultiPolygon":
        geojson["coordinates"] = [
            [list(map(lambda coord: coord[:2], ring)) for ring in polygon]
            for polygon in geojson["coordinates"]
        ]

    # Create an Earth Engine geometry object.
    ee_geometry = ee.Geometry(geojson)
    feature = ee.Feature(ee_geometry)
    ee_feature_collection = ee.FeatureCollection([feature])

    logger.info("AOI defined successfully.")
    return ee_feature_collection

In [94]:
geometry = load_aoi_from_shapefile(r"C:\Users\pc\My Drive\Uni\RAVI\Shapes\contorno_area_total.zip")

2025-07-24 12:03:23,394 - __main__ - INFO - AOI defined successfully.


In [95]:
# Define input parameters
# Geometry for ~100 hectares in Mato Grosso, Brazil (approx. 1km x 1km square)
geometry = ee.Geometry.Polygon(
    [[
        [-56.0, -12.0],
        [-55.989, -12.0],
        [-55.989, -12.009],
        [-56.0, -12.009],
        [-56.0, -12.0]
    ]]
)

start_date = '2021-01-01'
stop_date = '2021-12-31'


s1_collection = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filterBounds(geometry) \
    .filterDate(start_date, stop_date)

s1_collection.size().getInfo()  # This will trigger the processing and return the size of the collection

28

In [ ]:


processor = S1ARDImageCollection(
    geometry=geometry,
    start_date=start_date,
    stop_date=stop_date,
    polarization="VVVH",
    apply_border_noise_correction=True,
    apply_terrain_flattening=True,
    apply_speckle_filtering=True,
    output_format="DB"
)

collection = processor.get_collection()

collection.size().getInfo()  # This will trigger the processing and return the size of the collection